# Import and Settings
### Python - 3.9.10

In [2]:
import spotipy
import numpy as np
import pandas as pd
import seaborn as sb
from creds import cid, secret
import matplotlib.pyplot as mpl
from spotipy.oauth2 import SpotifyClientCredentials

In [3]:
#  Your Spotify API app credentials
# cid = 'your_client id'
# secret = 'your_client secret'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)


In [4]:
# Tops Songs 2020
# playlist_link = "https://open.spotify.com/playlist/37i9dQZF1EM9DVbOg6CjfI?si=f57f3dbd0c7c4342"

# Tops Songs 2021
# playlist_link = "https://open.spotify.com/playlist/2m7FyVPAKKwu3xfpn49miL?si=c4db6bb449f74d55"

# Tops Songs 2022
# playlist_link = "https://open.spotify.com/playlist/0rXAPXNnHgU3FUmBhKULXP?si=918768d5124a4a4c"

# Tops Songs 2023
playlist_link = "https://open.spotify.com/playlist/37i9dQZF1Fa1IIVtEpGUcU?si=ea1656c524ef4958"

playlist_URI = playlist_link.split("/")[-1].split("?")[0]

In [5]:
playlist_features_list = [
    "track_id",
    "track_name",
    "artist",
    "album",
    "tempo",
    "duration_ms",
    "danceability",
    "energy",
    "key",
    "loudness",
    "mode",
    "speechiness",
    "instrumentalness",
    "liveness",
    "valence",
    "time_signature"
    ]

df = pd.DataFrame(columns = playlist_features_list)
    
playlist = sp.user_playlist_tracks("spotify", playlist_URI)["items"]

for track in playlist:
    # Create empty dict
    playlist_features = {}
    # Get metadata
    playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
    playlist_features["album"] = track["track"]["album"]["name"]
    playlist_features["track_name"] = track["track"]["name"]
    playlist_features["track_id"] = track["track"]["id"]
    
    # Get audio features
    audio_features = sp.audio_features(playlist_features["track_id"])[0]
    for feature in playlist_features_list[4:]:
        playlist_features[feature] = audio_features[feature]
    
    # Concat the dfs
    track_df = pd.DataFrame(playlist_features, index = [0])
    df = pd.concat([df, track_df], ignore_index = True)
    
    
    results = pd.concat([df, track_df], ignore_index = True)

results

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
lil_graph_list = results[[
    "danceability",
    "energy",
    "speechiness",
    "instrumentalness",
    "liveness",
    "valence"
    ]]

big_graph_list = results[[
    "tempo",
    # "duration_ms",
    "key",
    "loudness",
    "mode",
    "time_signature"
    ]]

graph_list = results[[
    "tempo",
    "duration_ms",
    "danceability",
    "energy",
    "key",
    "loudness",
    "mode",
    "speechiness",
    "instrumentalness",
    "liveness",
    "valence"
    # "time_signature"
        ]]


graph_list

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# 
min_max_scaler = MinMaxScaler()

graph_list.loc[:]=min_max_scaler.fit_transform(graph_list.loc[:])


In [ ]:
# plot size
fig=mpl.figure(figsize=(12,8))

# convert column names into a list
categories=list(graph_list.columns)
# number of categories
N=len(categories)

# create a list with the average of all features
value=list(graph_list.mean())

# repeat first value to close the circle
# the plot is a circle, so we need to "complete the loop"
# and append the start value to the end.
value+=value[:1]
# calculate angle for each category
angles=[n/float(N)*2*3.14 for n in range(N)]
angles+=angles[:1]

# plot
mpl.polar(angles, value)
mpl.fill(angles,value,alpha=0.3)

# mpl.title('Discovery Weekly Songs Audio Features', size=35)

mpl.xticks(angles[:-1],categories, size=15)
mpl.yticks(color='black',size=15)
# mpl.style.use('dark_background') # Because dark mode everything
mpl.show()